# Computational Model of Spatial Auditory Attention

### Table of Contents
* [Basic model setup](cmsaa.ipynb#Basic-Model-Setup)
* * [Goal Map](cmsaa.ipynb#Goal-Maps)
* * [Saliency Map](cmsaa.ipynb#Saliency-Maps)
* * [Priority Map](cmsaa.ipynb#Priority-Maps)
* * [Attentional Bias](cmsaa.ipynb#Attentional-Bias)
* [Fitting model to the data](cmsaa.ipynb#Fitting-the-Priority-Map-to-the-Data)
* [180 Degree Model](cmsaa-180.ipynb#180-Degre-Data)
* * [180 Degree Data](cmsaa-180.ipynb#180-Degree-Data)
* * [180 Degree Results](cmsaa-180.ipynb#180-Degree-Results)
* [360 Degree Model](cmsaa-180.ipynb#180-Degre-Data)
* * [360 Degree Data](cmsaa-180.ipynb#180-Degree-Data)
* * [360 Degree Results](cmsaa-180.ipynb#180-Degree-Results)

In [15]:
import numpy as np
from scipy.optimize import curve_fit
from math import exp

## Basic Model Setup

### Goal Maps

#### Standard

Top-down attentional bias is represented as a gaussian curve, with the highest amount of attentional bias focused at the attended location. Less attentional bias is applied to locations further from the attended location.

In [16]:
class GoalMap:
    def __init__(self, attended_location):
        self.attended_location = attended_location
    
    def standard(self, x, mag, stdev):
        # gaussian equation
        return mag * np.exp(-abs(self.attended_location-x)**2 / (2*stdev**2))
    
    # shifts the goal map up or down by the minshift value
    def standard_minshift(self, x, mag, stdev, minshift):
        return minshift + standard(x,stdev, mag)
        

### Saliency Maps 

Bottom up attentional bias can be represented as an inverted gaussian curve or as a constant value across all locations.

#### Experimental

The saliency may be learned from the data using Instance Based Learning.

In [1]:
class SaliencyMap:
    def __init__(self, attended_location):
        self.attended_location = attended_location
    
    def standard(self, x, mag, stdev):
        # inverted gaussian equation
        return mag - mag * np.exp(-abs(self.attended_location-x)**2 / (2*stdev**2))
    
    # returns a constant attentional bias at every location in the map range
    def constant(self, x, value):
        return [value]*len(x)
    
    # returns a map where each degree location in the map range represents the probability of a sound at that location.
    # Probabilities are learned from the individual trials
    # Inputs: x - a list representing the range of locations that sounds can be presented from
    #         attended_location - represents the attended location for the current condition (probably -90,0,90 in the 180 degree case)
    #         trials - a 2d list, where each row contains [trial id, sound location and frequency] 
    # Outputs: saliency map - a list of size x. Each item in the list contains the probability that a sound will come from that location.
    #                         Probability is calculated using the equations found in Lejarraga 2010. https://onlinelibrary.wiley.com/doi/abs/10.1002/bdm.722 
    def ibl(self, x, attended_location, trials):
        
        saliency_map = []
        
        # TODO: replace this with IBL algorithm
        # - Calculate the activation for each trial in trials (Equation 3 in paper)
        # - For each location in x:
        # - - calculate the activation value as if that location were the next trial in trials (trial id + 1) (Equation 3 in paper)
        # - - calculate the probability of a sound coming from that location (Equation 2 in paper)
        # - - append probability to saliency_map.
        
        # given a 2d array of data (each row represents one trial)
        return saliency_map
        

### Priority Maps

The priority map represents the total attentional bias at a given location. It is calculated by adding the bias from the goal map to the bias supplied by the saliency map.

In [11]:
class PriorityMap:
    def __init__(self, attended_location):
        self.attended_location = attended_location
        
    def standard(self, x, gm_mag, gm_stdev, sm_mag, sm_stdev):
        
        gm = GoalMap(self.attended_location)
        sm = SaliencyMap(self.attended_location)
        
        self.goalmap = gm.standard(x, gm_mag, gm_stdev)
        self.saliencymap = sm.standard(x, sm_mag, sm_stdev)
        self.prioritymap = self.goalmap + self.saliencymap
        
        return self.prioritymap
        

### Attentional Bias

Attentional bias represents how much attention is available at each spatial location. It has an inverse relationship to reaction times, such that more attention leads to faster reaction times, and less attentional bias leads to slower reaction times. 

The mean reaction times from the data are converted to an attentional bias value using the function below.

In [12]:
def attentional_bias(expRTs):
    
    return (2000 - np.array(expRTs))/2000

## Fitting the Priority Map to the Data

The priority map is fit to the data using the curve_fit function available in the scipy library. 
curve_fit expects the function to be fit (the priority map), the range of values to fit, the data to fit the funtion to, initial parameter values and the bounds, or constraints on what the parameter values are allowed to be.

curve_fit returns a list of the optimal parameter values found and the estimated covariance.

In [13]:
def optimize_prioritymap(attended_location, x, y, init_vals, min_bounds, max_bounds):

    pm = PriorityMap(attended_location)

    (best_vals,covar) = curve_fit(pm.standard, x, y, p0=init_vals, bounds=(min_bounds,max_bounds))

    return best_vals                  

#### Calculating the Error

The root mean squared error between the optimized curve and the data can be found using the following function.

In [14]:
def rmse(xs,pm,experimental):
    error = 0
    
    i = 0
    for x in xs:
        error += (experimental[i] - pm[x]) ** 2
        i += 1
        
    return error